# Práctico 2: Calcular e interpretar medidas de centralidad de nodo en redes reales

# Inicialización

Como siempre, comenzamos poder installar las bibliotecas `IGraph` y `CairoCffi` (necesaria para visualizar grafos).

In [ ]:
!pip install python-igraph
!pip install cairocffi

Luego vamos a descargar algunos datasets

Datos del Club de Karate.

In [ ]:
!wget "https://raw.githubusercontent.com/prbocca/na101_master/master/homework_02_measures/karate.graphml" -O "karate.graphml"

 Datos de Blogs sobre el Sida.

In [ ]:
!wget "https://raw.githubusercontent.com/prbocca/na101_master/master/homework_02_measures/aidsblog.edgelist" -O "aidsblog.edgelist"

In [ ]:
import igraph as ig
import matplotlib.pyplot as plt
import random
import statistics
import pandas as pd
import numpy as np

#1) Análisis inicial

Vamos a seguir (en python) las secciones 4.1 y 4.2 del libro [SANDR].

Recomendamos su lectura en paralelo, para darle más contenido al trabajo de práctico. En lo que resta, agregaremos la nomenclatura [SANDR4.x.y] para referinos a la Sección 4.x.y del libro.

Empezamos por cargar el grafo y verificar algunas de sus propiedades.

In [ ]:
g_karate = ig.load("karate.graphml")

print(g_karate.summary())

In [ ]:
g_karate.vcount(), g_karate.ecount()

Es un grafo no dirigido con pesos en las aristas:

In [ ]:
g_karate.is_directed()

In [ ]:
g_karate.es[0].attributes()

Recordamos como visualizarlo.

In [ ]:
visual_style = dict()
visual_style["bbox"] = (400, 400)

#transformo numero de colores a paleta
id_gen = ig.datatypes.UniqueIdGenerator()
color_indices = [id_gen.add(value) for value in g_karate.vs['color']]
palette = ig.drawing.colors.ClusterColoringPalette(len(id_gen))
colors = [palette[index] for index in color_indices]
visual_style["vertex_color"] = colors 

ig.plot(g_karate, **visual_style)

Y podemos tener una tabla con todos los atributos de los vértices (y los ID de los vértices)

In [ ]:
g_karate.get_vertex_dataframe()

En el siguiente paso, le vamos a pedir que encuentre todos los vecinos del nodo con `label=9` y que encuentre las aristas correspondientes. 

Recomendamos usar las siguientes funciones.

In [ ]:
help(ig.VertexSeq.find)
help(ig.Graph.neighbors)
help(ig.Graph.get_eid)

In [ ]:
neighbors = None
edges = []

### START CODE HERE
### END CODE HERE

print(neighbors)
print(edges)

#2) Distribución de grado

Como primera de las herramientas para analizar el gráfo en su totalidad (a diferencia de un nodo en particular), vamos a mirar la distribución de grado. Esto es, un histograma de la frequencia de los grados de todos los vértices en el grafo [SAND4.2.1].

## 2.1) Graficar el histograma de la distribución de grado `g_karate`, utilizar la función `ig.Graph.degree()`.

In [ ]:
### START CODE HERE
### END CODE HERE


##2.2) Cálculo de la "fortaleza" del grafo

El concepto de fuerza es muy similar al de distribución de grado con una diferencia. En la distribución de grado, el grado se cálcula como la cantidad de aristas de cada vértice. Pero que ocurre si las aristas tienen peso?

En este caso, podemos usar la fortaleza y consecuentemente la distribución de la fortaleza [SAND4.2.1].

Graficar el histograma de la fortaleza de `g_karate`, utilizar la función `ig.Graph.strength()`.

In [ ]:
### START CODE HERE
### END CODE HERE


##2.3) Grado promedio de los vecinos en función del grado propio

Otra métrica que ayuda a describir la estructura de un grafo es entender que tan populares son los vecinos de un nodo [SAND4.2.1].

Por ejemplo: en un grafo estrella: el grado promedio de los vecinos de todos los nodos menos 1 es `n-1` mientras que el grado promedio del faltante es `1`.

Para cada nodo, calcula el promedio de los grados de sus vecinos.

In [ ]:
degree = g_karate.degree()

#lista donde se guarda el promedio del grado de los vecinos
avgerage_degree_neighbours = None

### START CODE HERE
### END CODE HERE


In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(degree, avgerage_degree_neighbours)
ax.plot(degree, degree, color='r', alpha=0.1)
ax.set_xlabel("Degree")
ax.set_ylabel("Neighbour Avg Degree")
ax.set_title("Average neighbor degree versus vertex degree")
plt.show()

Observar que es mucho más común que los vecinos tengan mayor grado en promedio que mi grado. A esto se le llama la "paradoja de la amistad", y es muy relevante para estudiar los efectos de las redes sociales:
* *Feld, Scott L. (1991), "Why your friends have more friends than you do", American Journal of Sociology, 96 (6): 1464–1477, doi:10.1086/229693, JSTOR 2781907, S2CID 56043992.*

Ver más en [Friendship paradox, from Wikipedia](https://en.wikipedia.org/wiki/Friendship_paradox).

#3) Medidas de centralidad

Habiendo trabajado con distribuciones relacionadas al grado de los vertices, nos movemos a trabajar con la centralidad de los nodos y como estos valores pueden usarse para describir el grafo [SANDR4.2.2].

Nos vamos a concentrar en las siguientes medidas:

* Grado
* Intermediación (Betweenness)
* Cercanía (Closeness)
* Valor Propio (Eigenvalue centrality)
* Page Rank
* Hub / Authority Score

## 3.1) Ranking de los vértices más importantes del grago  `g_karate` 

In [ ]:
degree = g_karate.degree()

betweeness = g_karate.betweenness()

closeness = g_karate.closeness()

eig_cent = g_karate.evcent(directed=False)

page_rank = g_karate.pagerank(directed=False)

hub = g_karate.hub_score()
authority = g_karate.authority_score()

In [ ]:
df = pd.DataFrame([degree, betweeness, closeness, eig_cent, page_rank, hub, authority]).T
df.columns = ["Degree", "Betweenness", "Closeness", "Eigenvalue Centrality", "Page Rank", "Hub", "Authority"]

In [ ]:
df.sort_values("Degree", ascending=False).head(10)

Obtener un dataframe con 5 filas donde cada fila tenga los vértices más importantes según cada medida de centralidad.

In [ ]:
### START CODE HERE
### END CODE HERE


In [ ]:
# Qué vertices aparecen en el top 5 de todas las medidas de centralidad

### START CODE HERE
### END CODE HERE


## 3.2) Observando la utilidad de hub/authority en la red de Blogs sobre el Sida

Comenzamos cargando la red [SANDR4.2.2].

In [ ]:
g_aids = ig.load("aidsblog.edgelist")

ig.summary(g_aids)

Calculamos las centralidades hub y authority.

In [ ]:
#guardamos los valores de la centralidad en
hub_aids = authority_aids = None

### START CODE HERE
### END CODE HERE

print(hub_aids)
print(authority_aids)

Visualizamos e interpretamos

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 8))

layout = g_aids.layout_kamada_kawai()
visual_style = {}
visual_style["layout"] = layout
visual_style["bbox"] = (500, 500)
visual_style["margin"] = 10

#Hubs
visual_style["vertex_size"] =10 * np.sqrt(hub_aids)
ax_ = ax[0]
ig.plot(g_aids, **visual_style, target=ax_)
_ = ax_.axis("off")
ax_.set_title("Hubs")

#Authorities
visual_style["vertex_size"] =10 * np.sqrt(authority_aids)
ax_ = ax[1]
ig.plot(g_aids, **visual_style, target=ax_)
_ = ax_.axis("off")
ax_.set_title("Authorities")

plt.show()

#4) Redes sociales reales

Para bajar a tierra nuestro análisis, y al mismo tiempo practicar hacerlo sobre datos reales, nos vamos a enfocar en un dataset extraido de Twitter.

Twitter permite acceder parcialmente a datos de la red utilizando una cuenta de
desarrollador gratuita. El 30/08/2018 a las 11.30am se descargaron los 5000 tweets más recientes sobre #Uruguay. 

## 4.1) Cargar y explorar los datos

In [ ]:
!wget "https://raw.githubusercontent.com/prbocca/na101_master/master/homework_02_measures/tweets_uru.csv" -O "tweets_uru.csv"

Esta vez comenzamos el análisis desde los datos crudos (y no desde el grafo).
Manipularemos los datos para obtener el grafo de twitter. Esto es lo habitual cuando trabajamos con datos reales.

Para esto, vamos a utilizar la biblioteca `pandas` la cual es ubiquita en el ecosistema de Python.

Empezamos por cargar el dataset y observar alguas características generales.

In [ ]:
df_tweets = pd.read_csv("tweets_uru.csv")

print(df_tweets.shape)
display(df_tweets.head())

In [ ]:
df_tweets.info()

In [ ]:
df_tweets.nunique()

El dataset tiene 17 columnas, las que resultan interesantes para este ejercicio son:

* `text`: el texto del tweet
* `screenName`: el usuario que envia el tweet
* `isRetweet`: si el tweet es un retweet o es un texto original. Nota: todos los tweets que son retweets tienen en el campo text: "RT @usuario_original: texto"
* `retweetCount`: cantidad de retweets que se hicieron sobre este tweet

In [ ]:
columns = ['text', 'screenName', 'isRetweet', 'retweetCount']

display(df_tweets[columns])

##4.2) Tweets más populares, y eliminación del SPAM.

Los tweets con más retweets parecen ser spam.

In [ ]:
df_tweets.sort_values("retweetCount", ascending=False).head(10)

Investiguemos más esos tweets

In [ ]:
fig, ax = plt.subplots(figsize=(16, 4))
ax.set_yscale("log")
df_tweets["retweetCount"].hist(bins=100, ax=ax)

Se observa que hay una gran separación en popularidad entre los tweets con unos pocos cientos de retweets, y los que tienen más de 15000 retweets.

Parece que podemos hacer un corte en 15000, siendo spam los que tienen más retweets. 

Observar que eliminamos 28 tweets (de spam).

In [ ]:
df_tweets = df_tweets[df_tweets["retweetCount"] < 15000]

print(df_tweets.shape)

Repetir el histograma de cantidad de retweets

In [ ]:
### START CODE HERE
### END CODE HERE


Mostrar los 5 tweets más populares (con más retweets) que no sean spam.

In [ ]:
### TIPs: ordenar los datos de acuerdo a la columna 'retweetCount'
### START CODE HERE
### END CODE HERE


##4.3) Crear la red de quién hace retweet de quién

Vamos a crear la red de quién hace retweet de quién. 

Por tanto no nos sirven los tweets sin retweets.
A continuación, procedemos a eliminarlos.

Además, vamos a eliminar los tweets con solo un retweet, sino la red quedaría muy densa.

Observar que eliminamos cerca de 1500 tweets (que no fueron reenviados o fueron reenviados solo una vez).

In [ ]:
df_tweets = df_tweets[df_tweets["retweetCount"] >= 2]

print(df_tweets.shape)

A continuación, le proponemos extraer una red a partir de estos datos. Para esto, vamos a crear una arista $e = (u,v)$ entre dos nodos $u$ y $v$ si $u$ retweeteo a $v$.

Nosotros usando una simple heurística encontramos 2964

In [ ]:
tweet_edges = None #dataframe con dos columnas "source" y "retweeter", con los nombres de usuarios de quien es el original del tweet y quien lo reenvio

### TIPs: solo para los tweets que son retweets, quedarse con el usuario que origina el tweet dentro del campo text
### START CODE HERE
### END CODE HERE

tweet_edges 

Una vez que tenemos las aristas, procedemos a crear el grafo dirigido de quién hace retweet de quién.

Este grafo tiene 2368 nodos y 2964 aristas.

In [ ]:
g_tweets = ig.Graph.TupleList(tweet_edges.itertuples(index=False), directed=True)

g_tweets.summary()

Una visualización con nombres de los vértices para un grafo tan grande es un gran desafío.

A continuación una visualización aceptable.

In [ ]:
random.seed(1234)
visual_style = dict()
visual_style["layout"] = g_tweets.layout_drl(options={'simmer_attraction':0})
visual_style["bbox"] = (1200, 1200)
visual_style["vertex_size"] = 3
visual_style["vertex_color"] = 'red'
visual_style["vertex_label"] = g_tweets.vs["name"]
visual_style["vertex_label_size"] = 4
visual_style["edge_width"] = 0.3
visual_style["edge_arrow_size"] = 0.1
ig.plot(g_tweets, **visual_style)

##4.4) Importancia de los usuarios (centralidad de vértices)


Como se llama el usuario con más retweets en la red.

Solución:  `jgamorin`.


In [ ]:
### START CODE HERE
### END CODE HERE


Podemos calcular las métricas de centralidad ya vistas y comprar los usuarios más populares de acuerdo a ellas.

Solución (ordenado de más a menos centralidad):

| betweeness    | hub            | authority       |
|---------------|----------------|-----------------|
| jgamorin      | jgamorin       | Nicomatute19    |
| Rubiia215     | emekavoces     | ElOjoChurrinche |
| YerbaSaraUy   | PabloLarraz10  | bugabea         |
| nacho_uriarte | MaurAntunez    | colombopp       |
| Cabrosa18     | Ciudadanos_MVD | juan37778491    |


In [ ]:
### START CODE HERE
### END CODE HERE


##4.5)(Opcional) Repetir con nuevos datos

Lamentablemente desde 2015, las principales redes sociales han cerrado sus APIs para acceder a los datos de redes (amigos, etc). Solo algunas tienen un acceso limitado (gratuito o con suscripción). El sitio SociLab (http://socilab.com/) realizaba un análisis básico de la red Linkedin del usuario. De forma excepcional (y por razones históricas) este sitio tuvo acceso a esta API hasta el 2018. 

Actualmente estan todas cerradas, solo existen muchas ofertas de servicios online, que utilizan tu cuenta de usuario para extraer la información (muy parcial) de las redes sociales. Ejemplos son: 
* https://netlytic.org/,
* https://mentionmapp.com/, 
* https://socioviz.net, etc. 

Los datos de la sección anterior se capturaron utilizando Netlytic con datos de twitter. La interfaz no es intuitiva, pero es potente. Los pasos son:
* i) crear una cuenta;
* ii) realizar un nuevo dataset vinculando tu cuenta de Twitter, escribiendo un nombre al dataset y las palabras de búsqueda (ej. “#Uruguay”); 
el resultado lleva unos minutos, y se puede acceder y en la sección “mi dataset”;
* iii) descargar el dataset en formato .csv;
* iv) una de las opciones de análisis es basado en redes, en donde puede visualizar la red y exportarla.

También pueden descargarse los datos utilizando librerias específicas. 
Por ejemplo, el paquete de `R` llamado `twitteR`, realiza la tarea: 
* i) crear una cuenta de desarrollo en https://developer.twitter.com/;
* ii) crear una aplicación de Twitter para obtener credenciales de acceso al API (consumer key, consumer secret, access token, access token secret);
* iii) usar el API desde R. 

Puede por ejemplo descargarse los 5000 tweets más recientes de #Uruguay (o del tópico que se desee) y repetir las partes anteriores de la Sección 4).
